In [0]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [0]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [0]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('Confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('Deaths')
dft_recovered = get_frame('Recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]

dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfc_recovered = dft_recovered.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()
dfp_recovered = dft_recovered.groupby(COL_REGION)[dt_5ago].sum()

In [0]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81439.0,3274.0,72814.0,81102,3241,69755,337.0,33.0,3059.0,4.0,Asia
1,Italy,59138.0,5476.0,7024.0,35713,2978,4025,23425.0,2498.0,2999.0,9.3,Europe
2,US,33276.0,417.0,178.0,7783,118,0,25493.0,299.0,178.0,1.3,North America
3,Spain,28768.0,1772.0,2575.0,13910,623,1081,14858.0,1149.0,1494.0,6.2,Europe
4,Germany,24873.0,94.0,266.0,12327,28,105,12546.0,66.0,161.0,0.4,Europe
5,Iran,21638.0,1685.0,7931.0,17361,1135,5389,4277.0,550.0,2542.0,7.8,Asia
6,France,16044.0,674.0,2200.0,9105,148,12,6939.0,526.0,2188.0,4.2,Europe
7,South Korea,8897.0,104.0,2909.0,8413,84,1540,484.0,20.0,1369.0,1.2,Asia
8,Switzerland,7245.0,98.0,131.0,3028,28,15,4217.0,70.0,116.0,1.4,Europe
9,United Kingdom,5741.0,282.0,67.0,2642,72,67,3099.0,210.0,0.0,4.9,Europe


In [0]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'Cases': 336004.0,
 'Cases (+)': 121094.0,
 'China Cases': 81439.0,
 'China Cases (+)': 337.0,
 'China Deaths': 3274.0,
 'China Deaths (+)': 33.0,
 'China Recovered': 72814.0,
 'China Recovered (+)': 3059.0,
 'Deaths': 14643.0,
 'Deaths (+)': 5910.0,
 'EU Cases': 169164.0,
 'EU Cases (+)': 78724.0,
 'EU Deaths': 8807.0,
 'EU Deaths (+)': 4796.0,
 'EU Recovered': 12789.0,
 'EU Recovered (+)': 7351.0,
 'Recovered': 98334.0,
 'Recovered (+)': 15139.0,
 'US Cases': 33276.0,
 'US Cases (+)': 25493.0,
 'US Deaths': 417.0,
 'US Deaths (+)': 299.0,
 'US Recovered': 178.0,
 'US Recovered (+)': 178.0,
 'since': Timestamp('2020-03-18 00:00:00'),
 'updated': Timestamp('2020-03-23 00:00:00')}

In [0]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,2,0,0,4,5,5,1,0,0,2,0,16,0
Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,2,11,10,5,4,9,4,4,5,6,6,13,0
Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,2,7,0,5,0,2,1,0,0,4,2,11,11,6,6,14,13,3,49,62,0
Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,37,0,14,22,13,25,0
Angola,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
